In [2]:
from myio import *

main_dir = Path('out_small')
#tools = ['mm2-fast', 'winnowmap']
#tools = ['shmap-k25-r0.05-t0.4-diff0.05', 'minimap', 'blend', 'mapquik', 'mm2-fast', 'winnowmap'] #, ['shmap-k25-r0.05-t0.4-diff2', 'shmap-k25-r0.05-t0.4-diff0.02', 'shmap-k25-r0.05-t0.4-diff0.15', 'blend'] #, 'shmap-diff0.02', 'shmap-diff0.1'] #, 'blend', 'winnowmap', 'minimap', 'mapquik', 'mm2-fast'] #, 'shmap-diff0.15', 'shmap-diff0.02', 'shmap'] #, 'shmap-MIN_DIFF-002']  # 'shmap-noprune',  'shmap-k25-r0.05-t0.4-diff0.15', 'shmap-k25-r0.05-t0.5-diff0.1'
#tools = ['shmap-k25-r0.05-t0.4-diff0.05',
#         'shmap-k25-r0.03-t0.5-diff0.05',
#         'shmap-k25-r0.05-t0.5-d0.15-o0.1-mfixed_C',
#         'shmap-k25-r0.05-t0.5-d0.15-o0.1-mbucket_SH',    
#         'shmap-k25-r0.05-t0.5-d0.15-o0.1-mbucket_LCS',    
#         'blend'] #['minimap']#, , 'shmap-k25-r0.05-t0.4-diff0.02']
tools = [
         'mapquik',
         'winnowmap',
         'minimap',
         'mm2-fast',
         'blend',
         'mashmap',
         #'shmap-k25-r0.01-t0.35-d0.075-o0.3-mbucket_LCS',
         #'shmap-k25-r0.01-t0.4-d0.2-o0.3-mContainment',
         'shmap-k25-r0.01-t0.4-d0.2-o0.3-mbucket_LCS',
         'shmap-k25-r0.01-t0.35-d0.075-o0.3-mJaccard',
         'shmap-k25-r0.01-t0.35-d0.075-o0.3-mfixed_C',
         'shmap-k25-r0.01-t0.4-d0.075-o0.3-mfixed_C',
         #'shmap-k25-r0.01-t0.4-d0.05-o0.3-mfixed_C',
         #'shmap-k25-r0.01-t0.4-d0.05-o0.2-mfixed_C',
         #'shmap-k25-r0.01-t0.4-d0.1-o0.2-mfixed_C',
         #'shmap-k25-r0.01-t0.5-d0.15-o0.1-mfixed_C',
         #'shmap-k25-r0.01-t0.5-d0.15-o0.1-mbucket_LCS',    
         #'shmap-k25-r0.01-t0.5-d0.15-o0.1-mbucket_SH',    
         #'shmap-k25-r0.01-t0.4-d0.15-o0.1-mfixed_C',
         #'shmap-k25-r0.01-t0.4-d0.15-o0.1-mbucket_SH',    
         #'shmap-k25-r0.01-t0.4-d0.15-o0.1-mbucket_LCS',    
         #'shmap-k25-r0.005-t0.4-d0.15-o0.1-mfixed_C',
         #'shmap-k29-r0.01-t0.4-d0.075-o0.1-mfixed_C',
         #'shmap-k25-r0.01-t0.4-d0.05-o0.1-mfixed_C',
         #'shmap-k23-r0.01-t0.4-d0.075-o0.1-mfixed_C',
         #'shmap-k25-r0.005-t0.35-d0.075-o0.3-mfixed_C',
         ] #['minimap']#, , 'shmap-k25-r0.05-t0.4-diff0.02']
experiments = [
    ('chrY',  'chrY-readschrY-a?-d2-l?'),
#    ('chm13', 'chm13-readshg002-a?-d1-l?'),
#    ('chm13', 'chm13-readschm13-a?-d1-l?'),
#    ('chm13', 'HG002_small'),
]

pd.set_option('display.width', 100)
css = """ <style> table { font-family: "Courier New", Courier, monospace; } </style> """
display(HTML(css))
dfs = []
keys = []
for refname, data in experiments:
    df = get_comparison_table(main_dir=main_dir, refname=refname, experiment=data, tools=tools).round(2)
    dfs.append(df)
    keys.append(data)
    df_styled = (
        df.style
        .set_caption(data)
        .set_table_styles( [{ 'selector': '.index_name, .row_heading', 'props': [('text-align', 'left')] }] )
        #.apply(highlight_min, subset=pd.IndexSlice[:, :]))
        #.apply(highlight_min, subset=pd.IndexSlice[:, df.columns.difference(['Mapped Q60'])])
        #.apply(highlight_max, subset=pd.IndexSlice[:, ['Mapped Q60']])
    )
    display(df_styled)
DF = pd.concat(dfs, keys=keys)
#display(DF)
DF.to_latex('evals-table.tex', escape=True, multirow=False)

Computing mashmap for chrY-readschrY-a?-d2-l?...
An error occurred while reading PAF out_small/mashmap/chrY-readschrY-a?-d2-l?/mashmap: list index out of range


,Mapped Q60,Q<60 or missed,Wrong Q60,Index [sec],Map [sec],Memory [GB]
mapquik,41,99.5%,32,0.4,0.8,2.1
winnowmap,3056,63.0%,0,4.4,5280.7,3.1
minimap,2733,66.9%,0,1.5,396.4,0.3
mm2-fast,2733,66.9%,0,1.4,391.3,0.3
blend,4097,50.4%,3,0.8,125.9,0.2
shmap-k25-r0.01-t0.4-d0.2-o0.3-mbucket_LCS,8248,0.2%,922,0.2,23.7,0.2
shmap-k25-r0.01-t0.35-d0.075-o0.3-mJaccard,4531,45.2%,4,0.2,34.0,0.2
shmap-k25-r0.01-t0.35-d0.075-o0.3-mfixed_C,3822,53.8%,0,0.2,15.2,0.2
shmap-k25-r0.01-t0.4-d0.075-o0.3-mfixed_C,3821,53.8%,0,0.2,13.5,0.2


In [44]:
import re

def read_sketching_evals(eval_sketching_dir, eval_experiment_str):
    sketching_df = pd.DataFrame()
    for full_dir in eval_sketching_dir.glob('shmap-*'):
        shmap_str = full_dir.name
        # sample dir=shmap-k25-r0.05-t0.5-d0.15-o0.1-mbucket_LCS
        params = shmap_str.split('-')
        d = { param[0]: param[1:] for param in params[1:] }
        try:
            times = read_times(full_dir/eval_experiment_str/shmap_str)
            idx_time, map_time, memory = times.loc[['Index [sec]', 'Map [sec]', 'Memory [GB]']]
            sketching_df.loc[d['k'], d['r']] = map_time
        except Exception as e:
            print(f"Error reading times for {shmap_str}: {e}")
            continue
        

    return sketching_df

def plot_SM_heatmap(df, name, title, cm):
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(df.astype(float), ax=ax, annot=True, fmt=".2f", cmap=cm) #, cbar=False)
    ax.set_title(title)
    ax.set_ylabel(df.index.name)
    ax.set_xlabel(df.columns.name)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()
    fig.savefig(f'imgs/{name}.png', dpi=300)
    plt.close(fig)

eval_sketching_dir = Path('out_small/sketching')
eval_experiment_str = 'chm13-chr1-readschm13-chr1-a?-d1-l?'
sketching_df = read_sketching_evals(eval_sketching_dir, eval_experiment_str)
display(sketching_df)
plot_SM_heatmap(sketching_df, name='heatmapSM_maptime', title='Map time', cm='mako_r')


/tmp/ipykernel_2272101/1496810121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sketching_df.loc[d['k'], d['r']] = map_time
/tmp/ipykernel_2272101/1496810121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4.6' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sketching_df.loc[d['k'], d['r']] = map_time
/tmp/ipykernel_2272101/1496810121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3.2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sketching_df.loc[d['k'], d['r']] = map_time


ValueError: not enough values to unpack (expected 2, got 0)

In [44]:
import re

def read_sketching_evals(eval_sketching_dir, eval_experiment_str):
    sketching_df = pd.DataFrame()
    for full_dir in eval_sketching_dir.glob('shmap-*'):
        shmap_str = full_dir.name
        # sample dir=shmap-k25-r0.05-t0.5-d0.15-o0.1-mbucket_LCS
        params = shmap_str.split('-')
        d = { param[0]: param[1:] for param in params[1:] }
        try:
            times = read_times(full_dir/eval_experiment_str/shmap_str)
            idx_time, map_time, memory = times.loc[['Index [sec]', 'Map [sec]', 'Memory [GB]']]
            sketching_df.loc[d['k'], d['r']] = map_time
        except Exception as e:
            print(f"Error reading times for {shmap_str}: {e}")
            continue
        

    return sketching_df

def plot_SM_heatmap(df, name, title, cm):
    fig, ax = plt.subplots(figsize=(6, 4))
    sns.heatmap(df.astype(float), ax=ax, annot=True, fmt=".2f", cmap=cm) #, cbar=False)
    ax.set_title(title)
    ax.set_ylabel(df.index.name)
    ax.set_xlabel(df.columns.name)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()
    fig.savefig(f'imgs/{name}.png', dpi=300)
    plt.close(fig)

eval_sketching_dir = Path('out_small/sketching')
eval_experiment_str = 'chm13-chr1-readschm13-chr1-a?-d1-l?'
sketching_df = read_sketching_evals(eval_sketching_dir, eval_experiment_str)
display(sketching_df)
plot_SM_heatmap(sketching_df, name='heatmapSM_maptime', title='Map time', cm='mako_r')


/tmp/ipykernel_2272101/1496810121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '14.9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sketching_df.loc[d['k'], d['r']] = map_time
/tmp/ipykernel_2272101/1496810121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '4.6' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sketching_df.loc[d['k'], d['r']] = map_time
/tmp/ipykernel_2272101/1496810121.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3.2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sketching_df.loc[d['k'], d['r']] = map_time


ValueError: not enough values to unpack (expected 2, got 0)